In [ ]:
%load_ext autoreload
%autoreload 2
# some basics
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import time
from progress.bar import IncrementalBar


# unconditonal Conditional GAN 
from gan.utils import reset_random_seeds
from cgan.generator import UnetGenerator
from cgan.discriminator import ConditionalDiscriminator
from cgan.criterion import GeneratorLoss, DiscriminatorLoss
from cgan.utils import Logger

# for visualization 3d model
from visual_3d import visual_3d

In [9]:
# set random seed
reset_random_seeds(77777)

In [10]:
# import training data - 100 channel facies model
ensemble_facies = np.load('Ensemble_facies.npy')
ensemble_perm = np.load('Ensemble.npy')


In [ ]:
plt.subplot(1,2,1)
plt.imshow(ensemble_facies[0,0])
plt.subplot(1,2,2)
plt.imshow(ensemble_perm[0,0])

In [ ]:
visual_3d(ensemble_facies[0])

In [13]:
# Preprocess data
ensemble_facies_pad = np.zeros((100, 8, 64, 64))
ensemble_perm_pad = np.zeros((100, 8, 64, 64))

for i in range(100):
    ensemble_perm_pad[i] = np.pad(ensemble_perm[i], ((0, 1), (2, 2), (2, 2)), 'reflect')
    ensemble_facies_pad[i] = np.pad(ensemble_facies[i], ((0, 1), (2, 2), (2, 2)), 'reflect')


In [14]:
# hyper-parameters
LEARNING_RATE = 0.0002
BATCH_SIZE = 25
EPOCHS = 501
EXP_NAME = './conditioanl_gan_run02_alpha_1'
LOG_FILENAME = 'training_log'

# rescale training data from -1 to 1
ensemble_facies_scaled = ensemble_facies_pad * 2 - 1
ensemble_log_perm_pad = np.log(ensemble_perm_pad)
ensemble_log_perm_pad[ensemble_log_perm_pad<4] = 4
ensemble_log_perm_min, ensemble_log_perm_max = ensemble_log_perm_pad.min(), ensemble_log_perm_pad.max()
ensemble_log_perm_sacled = (ensemble_log_perm_pad - ensemble_log_perm_min)/(ensemble_log_perm_max- ensemble_log_perm_min)*2 -1


In [18]:

total_num_of_data = ensemble_facies_scaled.shape[0]
inputs  = torch.tensor(ensemble_facies_scaled.astype(np.float32).reshape(total_num_of_data, 1, 8, 64, 64))
targets = torch.tensor(ensemble_log_perm_sacled.astype(np.float32).reshape(total_num_of_data, 1, 8, 64, 64))
dataset = TensorDataset(targets, inputs)
data_loader = DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [ ]:
torch.cuda.set_device(1)
print(f"Using GPU: {torch.cuda.current_device()}")


In [20]:
## Instantiate models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = UnetGenerator().to(device)
discriminator = ConditionalDiscriminator().to(device)

## define optimizer and loss
# optimizers
g_optimizer = torch.optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
# loss functions
g_criterion = GeneratorLoss(alpha=10)
d_criterion = DiscriminatorLoss()

In [ ]:
print('Start of training process!')
logger = Logger(exp_name = EXP_NAME, filename= LOG_FILENAME)
for epoch in range(EPOCHS):
    ge_loss=0.
    de_loss=0.
    start = time.time()
    bar = IncrementalBar(f'[Epoch {epoch+1}/{EPOCHS}]', max=len(data_loader))
    for x, real in data_loader:
        x = x.to(device)
        real = real.to(device)

        # Generator`s loss
        fake = generator(x)
        fake_pred = discriminator(fake, x)
        g_loss = g_criterion(fake, real, fake_pred)

        # Discriminator`s loss
        fake = generator(x).detach()
        fake_pred = discriminator(fake, x)
        real_pred = discriminator(real, x)
        d_loss = d_criterion(fake_pred, real_pred)

        # Generator`s params update
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        # Discriminator`s params update
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        # add batch losses
        ge_loss += g_loss.item()
        de_loss += d_loss.item()
        bar.next()
    bar.finish()  
    # obttain per epoch losses
    g_loss = ge_loss/len(data_loader)
    d_loss = de_loss/len(data_loader)
    # count timeframe
    end = time.time()
    tm = (end - start)
    logger.add_scalar('generator_loss', g_loss, epoch+1)
    logger.add_scalar('discriminator_loss', d_loss, epoch+1)
    if epoch % 250 == 0:
        logger.save_weights(generator.state_dict(), f'generator_{epoch}')
        logger.save_weights(discriminator.state_dict(), f'generator_{epoch}')
    print("[Epoch %d/%d] [G loss: %.3f] [D loss: %.3f] ETA: %.3fs" % (epoch+1, EPOCHS, g_loss, d_loss, tm))
    if epoch % 10 == 0:
        plt.figure(figsize = (10,10))
        fake = generator(x).detach().cpu().numpy().squeeze()
        for i_subplot in range(9):
            plt.subplot(3,3,i_subplot+1)
            plt.imshow(fake[i_subplot,0])
        plt.savefig(os.path.join(EXP_NAME,f"outcomes_epoch_{epoch:05}.pdf"))
        plt.close()
        
        plt.figure(figsize = (10,10))
        real = real.detach().cpu().numpy().squeeze()
        for i_subplot in range(9):
            plt.subplot(3,3,i_subplot+1)
            plt.imshow(real[i_subplot,0])
        plt.savefig(os.path.join(EXP_NAME,f"outcomes_epoch_{epoch:05}_ans.pdf"))
        plt.close()
logger.close()
print('End of training process!')

In [33]:
torch.cuda.set_device(0)

inputs  = torch.tensor(ensemble_facies_scaled.astype(np.float32).reshape(total_num_of_data, 1, 8, 64, 64)).to(device)
targets = torch.tensor(ensemble_log_perm_sacled.astype(np.float32).reshape(total_num_of_data, 1, 8, 64, 64)).to(device)

In [41]:
torch.cuda.set_device(0)
generator = generator.to(device)
fake = generator(inputs[:10]).detach().cpu().numpy().squeeze()
input_real = inputs[:10].detach().cpu().numpy().squeeze()
real = targets[:10].detach().cpu().numpy().squeeze()

In [ ]:
plt.figure(figsize = (10,10))
for i in range(3):
    plt.subplot(3,3,3*i+1)
    plt.title(f'Real Image - {i}')
    plt.imshow(input_real[i][0])

    plt.subplot(3,3,3*i+2)
    plt.title(f'Observation - {i}')
    plt.imshow(real[i][0])

    
    plt.subplot(3,3,3*i+3)
    plt.title(f'Generated Image - {i}')
    plt.imshow(fake[i][0]>0)